In [3]:
from fidelity_model import FidelityModel
import pandas as pd

In [45]:
data = pd.read_csv('dataset.csv')
synopses = {id: synopsis for id, synopsis in zip(data['id'], data['synopsis'])}
test_data = pd.read_csv("user_based_correlations_train.csv")
test_pairs = [[synopses[id1], synopses[id2]] for id1, id2 in zip(test_data['id1'], test_data['id2'])][:500]
test_labels = test_data['score'][:500]


In [46]:
from lambeq import spiders_reader, IQPAnsatz, AtomicType
from lambeq.backend.quantum import Ket, H, CX, Controlled, X, Id, Discard

def fidelity_pqc_gen(sentence_1, sentence_2):
    # # Turn into PQCs using bag-of-words
    sentence_1_diagram = spiders_reader.sentence2diagram(sentence_1)
    sentence_2_diagram = spiders_reader.sentence2diagram(sentence_2)
    ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1}, n_layers=1, n_single_qubit_params=3)
    iqp = ansatz(sentence_1_diagram @ sentence_2_diagram)
    control = Ket(0) >> H
    fidelity_pqc =  iqp @ control
    CCX = Controlled(Controlled(X, distance=-1), distance=-1)
    fidelity_pqc >>= CX @ Id(1) >> CCX >> CX @ Id(1) >> Discard() @ Discard() @ H  # Swap Test
    return fidelity_pqc

In [47]:
test_circuits = [fidelity_pqc_gen(sentence_1, sentence_2) for sentence_1, sentence_2 in test_pairs[:500]]

In [16]:
model = FidelityModel()
model.load("runs/500_train_500_val_500_epochs/best_model.lt")

In [48]:
calculated_labels = model(test_circuits)

In [66]:
import numpy as np
errors = [200*(prediction-real)/(prediction+real) for prediction, real in zip(calculated_labels, test_labels)]  # RPD As Percentage

In [69]:
np.mean(200*(calculated_labels-test_labels)/(calculated_labels+test_labels))

np.float64(-143.77864445768705)

In [67]:
np.mean(errors)

np.float64(-143.77864445768705)